In [562]:
import pandas as pd
import numpy as np



In [563]:
# Here we have our loading dataset and took in mind the headers
df = pd.read_csv('HIV DATABASE WHO.csv', encoding='latin1', header=0)
print(df.head())


    IndicatorCode                                          Indicator  \
0  HIV_0000000027  Reported number of HIV tests performed where r...   
1  HIV_0000000027  Reported number of HIV tests performed where r...   
2  HIV_0000000027  Reported number of HIV tests performed where r...   
3  HIV_0000000027  Reported number of HIV tests performed where r...   
4  HIV_0000000027  Reported number of HIV tests performed where r...   

  ValueType ParentLocationCode         ParentLocation Location type  \
0   numeric                EMR  Eastern Mediterranean       Country   
1   numeric                AFR                 Africa       Country   
2   numeric                EMR  Eastern Mediterranean       Country   
3   numeric                WPR        Western Pacific       Country   
4   numeric                AFR                 Africa       Country   

  SpatialDimValueCode                    Location Period type  Period  ...  \
0                 QAT                       Qatar        Year 

In [564]:
# Drop columns with no values
df_cleaned = df.dropna(axis=1, how='all')


In [565]:
# Basic statistics and column types
display(df_cleaned.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3104 entries, 0 to 3103
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IndicatorCode           3104 non-null   object 
 1   Indicator               3104 non-null   object 
 2   ValueType               3104 non-null   object 
 3   ParentLocationCode      3104 non-null   object 
 4   ParentLocation          3104 non-null   object 
 5   Location type           3104 non-null   object 
 6   SpatialDimValueCode     3104 non-null   object 
 7   Location                3104 non-null   object 
 8   Period type             3104 non-null   object 
 9   Period                  3104 non-null   int64  
 10  IsLatestYear            3104 non-null   bool   
 11  Dim1 type               3104 non-null   object 
 12  Dim1                    3104 non-null   object 
 13  Dim1ValueCode           3104 non-null   object 
 14  Dim2 type               3104 non-null   

None

In [566]:
# We are going to clean the "Value " column, handling missing values, outliers and non numeric values .
column_ad = 'Value'
if column_ad in df_cleaned.columns:
    # Remove spaces and non-numeric values
    df_cleaned.loc[:, column_ad] = df_cleaned[column_ad].astype(str).str.replace(',', '').str.replace(' ', '')
    df_cleaned.loc[:, column_ad] = pd.to_numeric(df_cleaned[column_ad], errors='coerce')  # Convert to numeric, setting errors to NaN
    
    df_cleaned.loc[:, column_ad] = df_cleaned[column_ad].astype(float)
    display(df_cleaned[column_ad].describe())
    
    display(df_cleaned[column_ad].isnull().sum())  # Showing how many missing values we have
    
else:
    print(f"Column {column_ad} not found. Check column names: {df_cleaned.columns}")


count     1158.0
unique     890.0
top          0.0
freq        14.0
Name: Value, dtype: float64

1946

In [567]:
# Handling missing values

# Drop rows with missing values
df_cleaned = df_cleaned.dropna(subset=[column_ad])



In [568]:
# In this step we need to drop some useless columns
columns_to_drop = [
    'ValueType', 'ParentLocationCode', 'Location type', 'SpatialDimValueCode',
    'Period type', 'Dim1 type', 'Dim1ValueCode', 'Dim2 type','IsLatestYear',
    'Dim2ValueCode', 'FactValueTranslationID', 'DateModified'
]

# Drop no functional clumns
df_cleaned = df_cleaned.drop(columns=[col for col in columns_to_drop if col in df_cleaned.columns])

In [569]:
#To facilitate the process of understanding the database
columns_to_rename = {
    'ParentLocation': 'Continent',
    'Location': 'Country',
    'Period': 'Year',
    'Dim1': 'Sex',
    'Dim2': 'Age'
}

# Rename columns if they exist in the dataset
df_cleaned = df_cleaned.rename(columns={col: new_col for col, new_col in columns_to_rename.items() if col in df_cleaned.columns})


In [570]:
#We will create a key that will help us with the next step 
def concatenate_columns(df):
    df["Concatenated"] = (
        df["Country"].str.replace(" ", "") + 
        df["Year"].astype(str) + 
        df["Sex"].str.replace(" ", "") + 
        df["Age"].str.replace(" ", "")
    )
    return df

# Apply the function
df_cleaned = concatenate_columns(df_cleaned)

# Show the first 6 rows to verify
df_cleaned[["Country", "Year", "Sex", "Age", "Concatenated"]].head(6)



,Country,Year,Sex,Age,Concatenated
0,Qatar,2023,Both sexes,<15 years of age,Qatar2023Bothsexes<15yearsofage
1,Rwanda,2023,Female,15+ years,Rwanda2023Female15+years
2,Iran (Islamic Republic of),2023,Female,15+ years,Iran(IslamicRepublicof)2023Female15+years
3,Malaysia,2023,Male,15+ years,Malaysia2023Male15+years
4,Benin,2023,Both sexes,All age groups (total),Benin2023BothsexesAllagegroups(total)
5,Tajikistan,2023,Both sexes,All age groups (total),Tajikistan2023BothsexesAllagegroups(total)


In [571]:
# Split the dataset based on the "Indicator" column
df_tested = df_cleaned[df_cleaned["Indicator"] == "Reported number of HIV tests performed where results were received by a person (testing volume)"]
df_positive_percentage = df_cleaned[df_cleaned["Indicator"] == "Percentage of HIV-positive results returned to people in the calendar year"]



In [572]:
# Merge the datasets using the "Concatenated" column
df_positive_percentage = df_positive_percentage.merge(
    df_tested[["Concatenated", "Value"]],  # Selecting only needed columns
    on="Concatenated",
    how="left"  # Left join to keep all rows from Positive Percentage
)



In [573]:
# Perform the calculation: (Integer Value * Value) / 100
df_positive_percentage["Positive Cases"] = (df_positive_percentage["Value_x"] * df_positive_percentage["Value_y"]) / 100


In [574]:
#Changing names
df_positive_percentage.rename(columns={"Value_y": "Tested Cases"}, inplace=True)

In [575]:
df_positive_percentage["Negative Cases"] = (df_positive_percentage["Tested Cases"] - df_positive_percentage["Positive Cases"])


In [576]:
print(df_positive_percentage.columns)

Index(['IndicatorCode', 'Indicator', 'Continent', 'Country', 'Year', 'Sex',
       'Age', 'FactValueNumeric', 'Value_x', 'Language', 'Concatenated',
       'Tested Cases', 'Positive Cases', 'Negative Cases'],
      dtype='object')


In [577]:
# Save original and cleaned datasets in a new Excel file
with pd.ExcelWriter('HIV_DATABASE_CLEANED.xlsx') as writer:
    df.to_excel(writer, sheet_name='Original Data', index=False)
    df_cleaned.to_excel(writer, sheet_name='Cleaned Data', index=False)
    df_tested.to_excel(writer, sheet_name='Tested Population', index=False)
    df_positive_percentage.to_excel(writer, sheet_name='Positive Percentage', index=False)
    

print("Cleaning completed. File saved as 'HIV_DATABASE_CLEANED.xlsx' with original and cleaned data.")


Cleaning completed. File saved as 'HIV_DATABASE_CLEANED.xlsx' with original and cleaned data.
